In [1]:
import numpy as np
import tifffile as tiff
from scipy.ndimage import gaussian_filter
from multiprocessing import Pool, cpu_count
import os
from tqdm import tqdm 

ModuleNotFoundError: No module named 'tifffile'

In [ ]:
# For df/f processing function for each frame
def process_frame(frame_idx, F0):
    frame = image_data[frame_idx]
    smoothed_frame = gaussian_filter(frame, sigma=sigma)
    normalized_frame = frame / smoothed_frame
    deltaF = normalized_frame - F0
    deltaF_over_F0 = deltaF / F0
    return deltaF_over_F0

In [ ]:
# load red output into python 
input_path = "/Users/johnmarshall/Documents/Analysis/SPRT_analysis/imagingData/SPRT_m1_d5_11_15_48_43/d5_11_15_48_43_red.tiff"
image_data = tiff.imread(input_path )

In [ ]:
image_data.shape

In [ ]:
image_data.dtype

In [ ]:
# Parameters
pixel_size_microns = 2.5
smoothing_scale_microns = 100
sigma = smoothing_scale_microns / (pixel_size_microns * 2.355)  # Adjusted sigma
print(f"Gaussian filter sigma: {sigma:.2f}")

In [ ]:
num_workers = cpu_count()  # Number of CPU cores for multiprocessing

# Load TIFF file
print(f"Image shape: {image_data.shape}, dtype: {image_data.dtype}")

# Compute F0 (mean across time for each pixel)
print("Computing F0 (time-averaged fluorescence)...")
F0 = np.mean(image_data, axis=0)

In [ ]:
# Define subset range
frame_start = 0  # Start frame index
frame_end = 1000  # End frame index (exclusive)
output_path = "/Users/johnmarshall/Documents/Analysis/SPRT_analysis/imagingData/SPRT_m1_d5_11_15_48_43/d5_11_15_48_43_red.tiff"
# Save the result as a multipage TIFF file
subset_data = image_data[frame_start:frame_end]

result_list = []
for frame_idx in tqdm(range(subset_data.shape[0]), desc="Processing frames"):
    result_list.append(process_frame(frame_idx))

# Convert result to a 3D numpy array
result_array = np.stack(result_list, axis=0)

In [ ]:
# Save the result as a multipage TIFF file
print(f"Saving result to {output_path}...")
tiff.imwrite(output_path, result_array.astype(np.float32), photometric='minisblack')

In [ ]:
# Perform the analysis using multiprocessing
print("Performing ΔF(t)/F0 analysis...")
with Pool(num_workers) as pool:
    result = pool.map(process_frame, range(image_data.shape[0]))

# Convert result to a 3D numpy array
result_array = np.stack(result, axis=0)

# Save the result as a multipage TIFF file
print(f"Saving result to {output_path}...")
tiff.imwrite(output_path, result_array.astype(np.float32), photometric='minisblack')

print("Analysis completed and file saved.")